# Make Image Sequences for ConvLSTM

In [1]:
working_dir = "/Volumes/Samsung_T5/WeatherNet_V2"
img_dir = working_dir + "/data_clean"

## Import Modules

In [2]:
import pandas as pd 
import numpy as np 

from PIL import Image

import os 
import glob 
import shutil

import matplotlib.pyplot as plt 
import matplotlib.image as mpimg

from tqdm import trange
from tqdm import tqdm

In [3]:
os.system("dot_clean .")
os.listdir(img_dir)

['flir', 'top', 'bottom']

## Make Dir for npy Data

In [4]:
os.mkdir(working_dir + "/npy_dataset")

In [5]:
flir_npy_path = working_dir + "/npy_dataset/flir"
top_npy_path = working_dir + "/npy_dataset/top"
bottom_npy_path = working_dir + "/npy_dataset/bottom"

os.mkdir(flir_npy_path)
os.mkdir(top_npy_path)
os.mkdir(bottom_npy_path)

## Declare Img Paths

In [6]:
flir_dir = img_dir + "/flir"
top_dir = img_dir + "/top"
bottom_dir = img_dir + "/bottom"

os.chdir(flir_dir)
os.system("dot_clean .")

os.chdir(top_dir)
os.system("dot_clean .")

os.chdir(bottom_dir)
os.system("dot_clean .")

os.chdir(working_dir)

In [7]:
# Check if files have all the same images
pic_names = os.listdir(flir_dir)
b = os.listdir(top_dir)
c = os.listdir(bottom_dir)

pic_names.sort()
b.sort()
c.sort()

if pic_names == c == b:
    print("equal")
else:
    print("not equal")

equal


## Get Dates

In [8]:
weather_data_path = working_dir + "/weather_data_15min/15_min_weather_prep.csv"
weather_df = pd.read_csv(weather_data_path)
weather_df.head()

,time_stamp,Month,Average 60 m temperature,Average 60 m wind speed,Vector-averaged 60 m wind speed,Vector-averaged 60 m wind direction,Standard deviation of 60 m wind direction,Total precipitaion for the period,Estimated heat flux,Estimated friction velocity,...,Average global irrradiation,Average net radiation,Estimated surface roughness length,day_month,Average 10 m vapor pressure,Average 10 m dew point temperature,hour_min,time_stamp.1,target,p_target_1
0,2020_01_01_00_15,1,-3.4994,584.35,578.00,263.98,8.4474,0.0,0.0,34.89,...,0.0,-52.595,7.61,1,0.418,-5.1122,00_15,2020_01_01_00_15,0,0
1,2020_01_01_00_30,1,-3.5981,622.76,616.97,263.45,7.8146,0.0,0.0,38.53,...,0.0,-52.977,9.21,1,0.414,-5.2493,00_30,2020_01_01_00_30,0,0
2,2020_01_01_00_45,1,-3.7045,642.28,636.82,263.51,7.4678,0.0,0.0,44.15,...,0.0,-53.749,17.02,1,0.414,-5.2413,00_45,2020_01_01_00_45,0,0
3,2020_01_01_01_00,1,-3.8625,596.84,591.08,262.92,7.9610,0.0,0.0,39.93,...,0.0,-54.059,14.98,1,0.411,-5.3480,01_00,2020_01_01_01_00,0,0
4,2020_01_01_01_15,1,-3.9970,559.11,554.50,262.78,7.3519,0.0,0.0,30.41,...,0.0,-53.716,4.33,1,0.407,-5.4834,01_15,2020_01_01_01_15,0,0


In [9]:
dates = weather_df["time_stamp"].values
print(dates[:5])

['2020_01_01_00_15' '2020_01_01_00_30' '2020_01_01_00_45'
 '2020_01_01_01_00' '2020_01_01_01_15']


## Make npy Samples

In [10]:
seq_len = 4
img_height = 480
img_width = 640

In [11]:
def make_img_seq(seq_pic_names, path, npy_path,img_height,img_width):
    pics = []
    for j,pic_name in enumerate(seq_pic_names):
        im = Image.open(path + "/" + pic_name + ".jpg")  
        if im.size != (img_height,img_width):
            im = im.resize((img_height,img_width))
        im2arr = np.array(im)    
        im2arr = im2arr.reshape(3,img_height,img_width)
        pics.append(im2arr) 
    pics = np.stack(pics) 
    np.save(npy_path+"/"+seq_pic_names[-1], pics)
    return pics

In [ ]:
pic_names  = [name.split(".")[0] for i,name in enumerate(pic_names)]
for i,pic_name in tqdm(enumerate(pic_names)):
    # Get index in dates of pic name
    idx = np.where(dates == pic_name)
    
    if idx[0].size != 0:
        idx = idx[0][0]
    
        # Check if previous seq_len-1 is present in images
        check = False
        seq_names = dates[idx - seq_len + 1: idx + 1]
        check = all(elem in pic_names for elem in seq_names)

        # If present, make samples 
        if check == True:
            make_img_seq(seq_names,flir_dir,flir_npy_path,img_height,img_width)
            make_img_seq(seq_names,top_dir,top_npy_path,img_height,img_width)
            make_img_seq(seq_names,bottom_dir,bottom_npy_path,img_height,img_width)


1030it [21:24,  1.46s/it]

## Show Sample of npy Img Sequence

In [ ]:
sample =np.load('/Volumes/Samsung_T5/WeatherNet_V2/npy_dataset/bottom/2020_01_23_01_30.npy')
sample.shape

In [ ]:
fig=plt.figure(figsize=(8,8))
columns = 4
rows = 1
for i in range(1, columns*rows +1):
    fig.add_subplot(rows, columns, i)
    plt.imshow(sample[i-1,:,:,:].reshape(img_width,img_height,3))
plt.show()